In [195]:
import os
import time
from __future__ import print_function
import keras
import tensorflow as tf
from keras.datasets import mnist
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential, load_model
from keras.callbacks import Callback, TensorBoard, ModelCheckpoint, CSVLogger
import matplotlib.pylab as plt

## TensorBoard Callback Helper Class

In [196]:
class Batched_TensorBoard(TensorBoard):

    def __init__(self, log_dir = './log_dir'):
        self.log_dir = log_dir
        self.batch_writer = tf.summary.FileWriter(self.log_dir)
        self.step = 0
        super().__init__(self.log_dir)


    def on_batch_end(self, batch, logs={}):
        """Called after every batch"""
        for name, value in logs.items():
            if name in ['acc', 'loss']:
                summary = tf.Summary()
                summary_value = summary.value.add() #Empty
                summary_value.simple_value = value.item() # 0.87 (Accuracy Value)
                summary_value.tag = name #if "acc", tag = "accuracy" for more defined tags on the tensorboard
                self.batch_writer.add_summary(summary, self.step) 
    
        self.batch_writer.flush()
        self.step += 1 # Iterated over every batch

## Initialize Globals

In [203]:
timestamp = time.strftime("%m-%d.%H:%M")
TENSORBOARD_PATH = './graphs/' + timestamp
CSV_DIR = './csv'
CSV_PATH = CSV_DIR + '/'+ timestamp + '.csv'
MODELS_ROOT = './models'
MODEL_DIR = MODELS_ROOT + '/' + timestamp
MODEL_PATH = MODEL_DIR + '/model.epoch.{epoch:02d}.hdf5'

In [204]:
if not os.path.exists(MODELS_ROOT):
    os.makedirs(MODELS_ROOT)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)
if not os.path.exists(CSV_DIR):
    os.makedirs(CSV_DIR)

## Load Data

In [205]:
# laod data
img_x, img_y = 28, 28

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)
print('data type:', x_train.dtype)

x_train shape: (60000, 28, 28)
y_train shape: (60000,)
x_test shape: (10000, 28, 28)
y_test shape: (10000,)
data type: uint8


In [206]:
'''
# un-comment when testing changes
# only trains on 100 instances
x_train = x_train[:100]
y_train = y_train[:100]
x_test = x_test[:100]
y_test = y_test[:100]
''''''

## Data Pre Processing

In [207]:
# reshape the data into a 4D tensor - (sample_number, x_img_size, y_img_size, num_channels)
# because the MNIST is greyscale, we only have a single channel - RGB colour images would have 3
x_train = x_train.reshape(x_train.shape[0], img_x, img_y, 1)
x_test = x_test.reshape(x_test.shape[0], img_x, img_y, 1)
input_shape = (img_x, img_y, 1)

In [208]:
# data values are in range [0, 255]
# convert to floats between [0, 1]
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [209]:
# convert labels to one hot encoding
# categorical_crossentropy loss below
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## Build / Compile Model

In [210]:
# resolve duplicate tensorboard computation graphs
from keras import backend as K  
K.clear_session()

In [211]:
# build CNN
model = Sequential()

# conv: 32 5x5 filters
# 28 x 28 x 1 --> 24 x 24 x 32
model.add(Conv2D(
    32,
    kernel_size=(5, 5),
    strides=(1, 1),
    activation='relu',
    input_shape=input_shape
))

# max pool:
# 24 x 24 x 32 --> 12 x 12 x 32
model.add(MaxPooling2D(
    pool_size=(2, 2),
    strides=(2, 2)
))

# conv: 64 5x5 filters
# 12 x 12 x 32 --> 8 x 8 x 64
model.add(Conv2D(
    64,
    kernel_size=(5, 5),
    activation='relu'
))

# max pool:
# 8 x 8 x 64 --> 4 x 4 x 64
model.add(MaxPooling2D(
    pool_size=(2, 2),
    strides=(2, 2)
))

# flatten for dense layer
model.add(Flatten())

# dense layer
model.add(Dense(1000, activation='relu'))

# softmax layer
model.add(Dense(num_classes, activation='softmax'))

# compile model
model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy']
)

## Define Training Callbacks

In [212]:
# define callbacks

# helper class for tracking accuracy
class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))
        
# AccuracyHistory callback stores per epoch accuracy on results.__dict__['history']
accuracyHistory = AccuracyHistory()

# TensorBoard callback writes computation graph and per batch loss
tensorboardCallback = Batched_TensorBoard(log_dir=TENSORBOARD_PATH)

# Model Checkpoint Callback saves model and weights after each epoch
modelCheckpointCallback = ModelCheckpoint(filepath=MODEL_PATH)

# CSV Checkpoint writes accuracy and loss to csv file after each epoch
csvLogCallback = CSVLogger(filename=CSV_PATH)

## Train Model

In [213]:
# define hyper params
batch_size = 128
num_classes = 10
epochs = 10

validation_data = (x_test, y_test)

results = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=validation_data,
    callbacks=[history, tensorboardCallback, modelCheckpointCallback, csvLogCallback]
)

Train on 100 samples, validate on 100 samples
Epoch 1/10
100/100 [==============================] - 1s 11ms/step - loss: 2.2893 - acc: 0.1100 - val_loss: 2.1647 - val_acc: 0.2400
Epoch 2/10
100/100 [==============================] - 1s 9ms/step - loss: 2.1038 - acc: 0.3400 - val_loss: 2.0217 - val_acc: 0.4100
Epoch 3/10
100/100 [==============================] - 1s 9ms/step - loss: 1.9043 - acc: 0.5000 - val_loss: 1.8295 - val_acc: 0.5700
Epoch 4/10
100/100 [==============================] - 1s 9ms/step - loss: 1.6588 - acc: 0.6600 - val_loss: 1.5880 - val_acc: 0.6400
Epoch 5/10
100/100 [==============================] - 1s 9ms/step - loss: 1.3666 - acc: 0.8100 - val_loss: 1.3449 - val_acc: 0.6500
Epoch 6/10
100/100 [==============================] - 1s 9ms/step - loss: 1.0746 - acc: 0.8800 - val_loss: 1.1043 - val_acc: 0.7200
Epoch 7/10
100/100 [==============================] - 1s 9ms/step - loss: 0.7952 - acc: 0.9300 - val_loss: 0.9147 - val_acc: 0.7100
Epoch 8/10
100/100 [=========

## Load Saved Model

In [150]:
new_model = load_model('./models/09-27.22:37model.01.hdf5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              1025000   
_________________________________________________________________
dense_2 (Dense)              (None, 10)                10010     
Total para